# Af$\rho = \frac{4\Delta^2 r^2 I_d}{\rho I_s}$

In [5]:
## Afp

import numpy as np
import astropy.io.fits as pf
import photutils as phot
import matplotlib.pyplot as plt
import glob
%matplotlib inline

# From MPC Ephemeris
# Delta (Geocentric distance) (AU)
# Needs to be converted to m
Delta = 0.5

# r (Heliocentric distance) (AU)
r = 1.4585

# Solar Intensity at 1AU (W m-2 Hz-1)
# V
I_vs = 1.8076e-12
# R
I_rs = 2.1308e-12
# I
I_is = 2.4174e-12

# Convert AU to m
def AUm(AU):
    return AU*149597870700

Delta = AUm(Delta)

# Aperture Photometry

# Load file
im_loc = glob.glob("C:/Users/dirmu/Documents/fieldtrip/AllSpeed_45P/red_O170320_0060.fits")
image1 = pf.getdata(im_loc[0])

# Define centre of aperture
position = [(947.59333,1015.8933)]

SNR_arr = []

rad = range(1,20,1)

# Loop to find peak SNR
for r_i in rad:
    # Define aperture
    aperture_ob = phot.CircularAperture(position, r=r_i)

    # Donut aperture for background subtraction
    aperture_bkg = phot.CircularAperture(position, r=30)

    apers = [aperture_ob, aperture_bkg]

    # Flux information
    phot_table = phot.aperture_photometry(image1,apers)

    bkg_mean = phot_table['aperture_sum_1']/aperture_bkg.area()

    bkg_sum = bkg_mean*aperture_ob.area()
    final_sum = phot_table['aperture_sum_0'] - bkg_sum
    phot_table['residual_aperture_sum'] = final_sum
    SNR = (final_sum)**1/2
    SNR_arr.append(final_sum)

max_SNR = max(SNR_arr)
aper_size = rad[SNR_arr.index(max_SNR)]
print("Aperture size for max S/N =",aper_size)
# print(phot_table)

I_dust_ap = phot.CircularAperture(position, aper_size)
I_dust = phot.aperture_photometry(image1, I_dust_ap)
print("Dust flux through {} aperture = {}".format(aper_size,int(I_dust['aperture_sum'])))
I_d = int(I_dust['aperture_sum'])

# Size of aperture at comet

# Field is 10'
# Which is 2048 pixels
# 1 pixel = 10/2048  = 0.3''/pixel
# Size of pixel at distance d
d = Delta
rho = d*np.tan((0.3/3600)*(np.pi/180))/2
# Convert to cm
rho = rho*100
print("Size of aperture at comet =",rho,"m")


Afp = (4*(Delta**2)*(r**2)*I_d)/(I_vs*rho)
print("Afp is =",Afp)

Aperture size for max S/N = 18
Dust flux through 18 aperture = 11809
Size of aperture at comet = 5439532.07852 m
Afp is = 5.71758906754e+31


In [6]:
print(rho)

5439532.07852


In [25]:
# Size of aperture at comet

# Field is 10'
# Which is 2048 pixels
# 1 pixel = 10/2048  = 0.3''/pixel
# Size of pixel at distance d
d = Delta
size = d*np.tan((0.3/3600)*(np.pi/180))/2
print(size)

54395.3207852
